<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install DotMap

In [1]:
from dotmap import DotMap
import jax.numpy as jnp
import jax
import vsc
import trust_ncg as tncg
import che_tools as che

In [2]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [3]:
def model(c):
    c=DotMap(c)
    Ltot = c.Ftot - c.Vtot
    V = c.Vy * c.Vtot
    L = c.Lx * Ltot
    F = c.Fz * c.Ftot
    mass_balance = (F - V - L)
    mass_balance = (mass_balance/jnp.sum(F))**2

    FH = p.Hmix(0., F, c.FT)
    VH = p.Hmix(V, 0., c.flashT)
    LH = p.Hmix(0., L, c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    fugacity = (c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT)
                - c.Vy*c.flashP)
    fugacity = (fugacity/jnp.sum(c.Vy*c.flashP))**2

    return jnp.sum(mass_balance + energy_balance + fugacity)

In [4]:
s=DotMap()
s.Ftot=10
s.Fz = jnp.array([1/3, 1/3, 1/3])
s.FT = 450

s.flashP= 101325
v=DotMap()
v.Vy = vsc.Comp(s.Fz)
v.Lx = vsc.Comp(s.Fz)
v.flashT = vsc.Range(360, 300, s.FT)
v.Vtot = vsc.Range(s.Ftot/2, 0., s.Ftot)

/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [5]:
c=vsc.VSC(v,s)
model(c.xtoc(c.x))

DeviceArray(0.04658206, dtype=float64)

In [6]:
modelf = jax.jit(c.transform(model))
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

0:6, f: 0.046582062928938066
1:5, f: 0.0030325209129333807
2:5, f: 3.826376067593822e-05
3:5, f: 9.734012906658525e-09
Final results:
f: 7.292863401690084e-16
x: [-0.14989821 -0.10167514 -0.41635314  0.23361766  0.16785475 -0.60540527]


In [7]:
v_sol = c.xtoc(x_min)
df=vsc.todf(v_sol)
df

vector1   vector3                    
              1         1         2         3
FT          450                              
Ftot         10                              
Fz               0.333333  0.333333  0.333333
Lx               0.311418  0.326803  0.361779
Vtot     3.9739                              
Vy               0.366567  0.343236  0.290198
flashP   101325                              
flashT  352.966